# Chapter 6. Odds and Addends
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap06.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import DataFrames as Dfs
import Distributions as Dsts
import CairoMakie as Cmk

In [ ]:
include("./pmf.jl")
import .ProbabilityMassFunction as pmf

## Odds

Odds [odds = $\frac{P(success)}{P(failure)} = \frac{P(success)}{1 -
P(success)}$], are used in the form of so called **Bayes's Rule**.

Bayes’s Rule is convenient if you want to do a Bayesian update on paper or in
your head. It also sheds light on the important idea of evidence and how we can
quantify the strength of evidence.

In [ ]:
function getOdds(prob::Float64)::Float64
    @assert (0 <= prob <= 1) "prob takes value in range [0-1]"
    return prob / (1 - prob)
end

In [ ]:
# if prob of winning is 75% then the odds are
# 3 to 1
getOdds(0.75)

In [ ]:
# odds in favor (1 to 9)
getOdds(0.1)

In [ ]:
# we may prefer to report odds agains 9:1
getOdds(1-0.1)

We can also do the opposite transformation, below my (BL) convesion for practice

$odds = \frac{p}{1-p}$

$\frac{p}{1-p} = odds$

$\frac{p}{1-p} = odds$ # multiply both sides by (1-p)

$p = odds * (1-p)$ # multiply: odds * (1-p)

$p = odds - odds * p$ # move (odds*p) to the left with the opposite sign

$p + (odds * p) = odds$ # rewrite left side to different equivalent form

$p * (1 + odds) = odds$ # move (1+odds) to the right with the opposite sign

$p = odds / (1 + odds)$

Below, a Julia's function for that calculation:

In [ ]:
function getProb(odds::T)::T where T<:Union{Rational, Float64}
    return odds / (odds + 1) 
end

In [ ]:
getProb(3/2),
getProb(3//2)

In [ ]:
function getProb(yes::Int, no::Int)::Float64
    return yes / (yes + no) 
end

In [ ]:
getProb(3, 2)

[...] some computations are easier when we work with odds, [...], and some
computations are even easier with log odds, [...].

## Bayes's Rule

So far we have worked with Bayes’s theorem in the “probability form”:

$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$

Writing $odds(A)$ for odds in favor of `A`, we can express Bayes’s Theorem in “odds form”:

$odds(A|D) = odds(A) * \frac{P(D|A)}{P(D|B)}$

In general Bayes's Rule says that the posterior odds are prior odds times the likelihood ratio.


**My comment**

Not sure how this odds form came to be all of a sudden?!

$P(D|B)$ suggests that probability of failure was introduced, since
in the case of two exhaustive mutually exclusive events (success and failure we got):

$P(success) + P(failure) = 1$ or $P(A) + P(B) = 1$

therefore

$P(success) = 1 - P(failure)$ or $P(A) = 1 - P(B)$

So if:

$odds(A) = \frac{P(A)}{1 - P(A)} = \frac{P(A)}{P(B)}$, then

$odds(A|D) = \frac{P(A|D)}{1 - P(A|D)} = \frac{P(A|D)}{P(B|D)}$

Moreover, at the onset the Bayes's theorem had to be written in the $P(A|D)$ and not $P(H|D)$ form.

So:

$P(A|D) = \frac{P(A)P(D|A)}{P(D)}$, instead of

$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$

Given the above and based on the definition of `getProb(odds)` above we see that:

$P(A) = \frac{odds(A)}{1 - odds{A}} = \frac{odds(A)}{odds{B}}$, therefore we can rewrite

$P(A|D) = \frac{P(A)P(D|A)}{P(D)}$ as

$\frac{odds(A|D)}{odds(B|D)} = \frac{P(A)P(D|A)}{P(D)}$, so i we multiply both sides by $odds(B|D)$ we get

$odds(A|D) = odds(B|D) * \frac{P(A)P(D|A)}{P(D)}$, but I am not sure what happens next.

Anyway, let's get back to the cookie problem:

> Suppose there are two bowls of cookies. Bowl 1 contains 30 vanilla cookies and
> 10 chocolate cookies. Bowl 2 contains 20 of each. Now suppose you choose one of
> the bowls at random and, without looking, select a cookie at random. The cookie
> is vanilla. What is the probability that it came from Bowl 1?

Let's use the **Bayes's Rule**:

$odds(A|D) = odds(A) * \frac{P(D|A)}{P(D|B)}$

In [ ]:
# odds(A) = odds(bowl1) = p(bowl1)/p(bowl2)
cookiePriorOdds = 1 # 0.5/0.5
# p(D|A) = p(vanilla|bowl1) = 30/40 = 3/4 = 0.75
# p(D|B) = p(vanilla|bowl2) = 20/40 = 2/4 = 0.5
# likelihood ratio = p(D|A)/p(D|B)
cookieLikelihoodRatio = 0.75 / 0.5
cookiePostOdds = cookiePriorOdds * cookieLikelihoodRatio

In [ ]:
cookiePostProb = getProb(cookiePostOdds)

If we draw another cookie and it's chocolate, we can do another update:

In [ ]:
cookieLikelihoodRatio = (10/40) / (20/40)
cookiePostOdds *= cookieLikelihoodRatio

In [ ]:
cookiePostProb = getProb(cookiePostOdds)